## 모듈 불러오기

In [3]:
import time
import pandas as pd
import os
from selenium import webdriver
# pip install selenium 
from bs4 import BeautifulSoup as bs

## selenium webdriver 불러오기

In [142]:
driver = webdriver.Chrome('chromedriver_win32/chromedriver.exe')

## 로그인

In [143]:
driver.get('https://nid.naver.com/nidlogin.login')
# id & pw 입력 
driver.find_element_by_id('id').send_keys('아이디') #네이버 아이디 
driver.find_element_by_id('pw').send_keys('패스워드') #네이버 패스워드
# click
driver.find_element_by_xpath('//*[@id="frmNIDLogin"]/fieldset/input').click()

## 게시물 크롤링

In [184]:
# 저장용 데이터프레임, 카페 고유번호, 자유게시판 고유번호, 시작 페이지 번호, 끝 페이지 번호
def navercafe_crawl(club_id, board_id, page_start=1, page_end=10):
    df = pd.DataFrame(columns = ['date', 'title', 'content'])
    id_lst = []
    for page_num in range(page_start, page_end+1): 
        # 게시판 한 페이지 url
        board_url = 'https://cafe.naver.com/ArticleList.nhn?search.clubid={}&search.menuid={}&search.boardtype=L&search.page={}'.format(club_id, board_id, page_num)
        # url로 이동
        driver.get(board_url)
        # 게시물 링크 확보
        driver.switch_to.frame('cafe_main')
        post_lst = driver.find_elements_by_css_selector('div.inner_list > a.article')
        post_urls = [post.get_attribute('href') for post in post_lst if 'specialmenutype' not in post.get_attribute('href')]
        # 게시물 크롤링
        for url in post_urls:
            # 중복되는 게시물 제외
            post_id = url.split('&')[-2].split('=')[-1]
            if post_id in id_lst:
                continue
            id_lst.append(post_id)
            # 제목, 내용 크롤링
            try:
                driver.get(url)
                time.sleep(3)
                driver.switch_to.frame('cafe_main')

                row = {}
                
                # 게시일 추출
                date = driver.find_element_by_class_name('date').text
                row['date'] = date
                # 제목 추출
                title = driver.find_element_by_class_name('title_text').text
                row['title'] = title
                print('제목: ' + title)

                # 내용 추출
                content = driver.find_element_by_class_name('se-main-container').text
                row['content'] = content
                print('내용: ' + content)
                print('-'*100)
                df = df.append(row, ignore_index=True)
            except:
                pass
    # 데이터프레임으로 리턴
    return df

In [185]:
# 맥쓰는사람들 = 12175294, 자유게시판 = 15
club_id = 12175294   # 카페 고유번호
board_id = 15       # 카페 내 자유게시판 고유번호

In [186]:
# 결과물을 데이터프레임에 저장
mac_user_df = navercafe_crawl(club_id, board_id, page_start=1, page_end=2)

['2372407']
제목: 인텔 맥북에어 2020 중고 구매했슴다😂😂
내용: 2015년도에 
맥북에어2014 와 엘지 그램을 함께 구매했었는데
작년쯤부터 그램은 잠깐만 돌려도 소음이랑 발열이 발열이....
진짜 핫팩 같았어요...🔥🔥
저는 집에서 간단한 문서작업(ms), pc카톡, 인터넷 정도밖에 안하는데도요...... 
그래서 맥북의 위력을 깨달았습니다. 
얘는 냉랭해도 이렇게 냉랭할 수가 없어요. 
신랑이 들고 댕기다가 떨구기도 해서 흠집까지 있는데도ㅋㅋㅋ 와따입니다👍🏻👍🏻
여튼 m1 맥북을 너무 사고 싶긴 했는데 
걍 인텔 맥북 에어 2020 중고로 질렀어요. 
아이들을 키우고 있어서 정부 관련 홈페이지에서 처리해야할 일도 많은데
맥os에서는 안되는 홈페이지가 많더라구요ㅠㅎ
부트캠프를 써야할 것 같아서...ㅎㅎ
이것으로 또 5년을 한번 잘 버텨보렵니다💪🏻💪🏻
----------------------------------------------------------------------------------------------------
['2372407', '2372406']
제목: 오늘도 글렀네요❤️
내용: 프로는 오늘도 출고중❤️
----------------------------------------------------------------------------------------------------
['2372407', '2372406', '2372405']
제목: 애플월렛 시껍하게 만드네요 캘린더 연동때문에
내용: 오전에 아시아나 비행티켓 발권하고 했어요
관두고 아예 고향에 내려가려고요
모바일체크인하고 애플월렛에 담기 메뉴가 있길래 담아놓고
나중에 봤더니 
팀원들과 공유하는 계정의 구글 캘린더에 추가가 되어있는 겁니다 
놀래서 아이폰 기본 캘린더 앱에서 해당 일정을 얼른 지우고 맘 놓고 있었죠
오후에 확인해보니
그 탑승일정에 가로줄만 있는채로 일정이 남아있었더군요(오후 늦게야 발견했습니다)
아시아나 OZ8900 00월 00일 00시 탑승


['2372407', '2372406', '2372405', '2372396', '2372393', '2372392', '2372390', '2372389', '2372388', '2372387', '2372385', '2372382', '2372379', '2372377', '2372375', '2372374']
제목: 내가 맥을 사용하는 두번째 이유(마지막)는 OSX의 코어그래픽스(Quartz) 때문
내용: 내가 맥을 사용하는 두번째 이유(마지막)는 OSX의 코어그래픽스(Quartz) 때문.
CoreGraphics는 OSX이 처음 등장할때 부터 발전되온 기술인데
코어 2D, 코어 GL, 코어 오디오, 코어 디스플레이, 코어 프레임웍 등등을 포함하죠.
코어그래픽스 덕분에 맥오에스에선 화면에 보이는 모든것을 pdf로 내보내기가 가능하며(pdf 편집기 없이도)
가려진 레이어(포커스를 잃은)도 온전히 화면캡쳐가 가능해지는 등 
윈도우즈에서 불가능한 캡쳐 및 내보내기가 가능함.
또한 코어그래픽스 덕분에 돌고있는 앱을 독에 미션콘트롤로 축소하거나 아이콘화 해놓아도 
아이콘속에서 계속 돌아가는걸 보는게 가능해 짐.  
맥오에스에서 화면을 캡쳐하면 그림자 까지 캡쳐가 되면 복붙하면 그림자가 살아있는것도 
코어그래픽스 덕분인데, 캡쳐한걸 윈도서버에 보냈다가 되가져오면 그림자가 망가짐.
윈도10이 보여주는 창의 그림자와 맥오에스의 창의 그림자는 
보기엔 비슷해 보여도 차원이 다른 기술임. 
----------------------------------------------------------------------------------------------------
['2372407', '2372406', '2372405', '2372396', '2372393', '2372392', '2372390', '2372389', '2372388', '2372387', '2372385', '2372382', '2372379', '2372377', '2372375', '2372374

In [191]:
# 엑셀 파일로 저장
mac_user_df.to_excel('macuser_cafe.xlsx', index=False, encoding='cp949')

In [ ]:
# csv 파일로 저장
# mac_user_df.to_csv('macuser_cafe.csv', index=False, encoding='utf-8')

In [193]:
# 저장된 파일 확인
pd.read_excel('macuser_cafe.xlsx', encoding='cp949')
# pd.read_csv('macuser_cafe.csv', encoding='utf-8')

,date,title,content
0,2020.12.18. 18:03,인텔 맥북에어 2020 중고 구매했슴다😂😂,2015년도에 \n맥북에어2014 와 엘지 그램을 함께 구매했었는데\n작년쯤부터 그...
1,2020.12.18. 18:03,오늘도 글렀네요❤️,프로는 오늘도 출고중❤️
2,2020.12.18. 18:03,애플월렛 시껍하게 만드네요 캘린더 연동때문에,오전에 아시아나 비행티켓 발권하고 했어요\n관두고 아예 고향에 내려가려고요\n모바일...
3,2020.12.18. 17:34,애플워치6 알루미늄 -> 4 스댕,저렇게 넘어가는건 어떻게들 생각하십니까 행님들\naod 너무 잘쓰고 있긴한데 스댕이...
4,2020.12.18. 17:27,mid 2007 아이맥 20인치 - 무슨 용도로 쓸까요?,2007년 아이맥 입니다. \n한동안 안쓰다 다시 켜봤는데 엄청 느리네요 ㅠㅠ\n너...
5,2020.12.18. 17:23,"인케이스 신규 맥북 케이스 13, 16인치",인케이스에서 맥북 케이스랑 가방 새로 나왔는데\n이쁘더라고요~\n신규가입 20%세일...
6,2020.12.18. 17:20,아이폰12 데드픽셀 환불 후 재구매할지 고민입니다,안녕하세요.\n아이폰12 공홈에서 배송받아 사용중입니다. 모든 기능 정상 작동하고 ...
7,2020.12.18. 17:16,에어팟 프로 구형 리콜 받는 방법,블랙 컨슈머 분들이 이용 하실거 같아서 올리기 망설여졌는데\n지금까지 노캔 불량으로...
8,2020.12.18. 17:16,에어팟 맥스는,‘일반인 기준으로 고음질’\n그 이상도 이하도 아니겠네요\n에어팟(맥스)의 헤드폰 ...
9,2020.12.18. 17:16,빅서 업데이트후 보드 손상! 공유,안녕하세요~ 2018맥북 프로 13인치 터치바모델 2018년 12월에 구매해서 카탈...
